# Evaluation of the neighborhoods in Oslo, Norway in order to develop a business activity.

## I. Introduction

The development of businesses brings diverse commercial activities of interest such as capital investment, job creation and increases the activity of the area, often improving the quality of life of the inhabitants in the area. However, it is necessary that the business activity to be developed is economically profitable and sometimes the best option is to choose a neighborhood that already has an activity developed at least to some extent. Therefore, it is important to evaluate the presence of commercial activity in the different neighbourhoods of Oslo. 

## II. Data

In order to obtain the data we need to start aur data analysis, we need to import all  the necessaty libraries.

### Import necessary Libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

### Define Foursquare Credentials and Version
#### Make sure that you have created a Foursquare developer account and have your credentials handy

In [4]:
!pip install pgeocode
import pgeocode
print('library pgcode imported!')

library pgcode imported!


## Obtaining boroughs latitudes


In order to obtain the boroughs latitudes we need to precisely know which are the boroughs in Oslo. For this purpose we will scrap a Wikipedia weppage containing the information we are looking for in Oslo.

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [6]:
url='https://en.wikipedia.org/wiki/List_of_boroughs_of_Oslo'

In [7]:
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [63]:
table

<table class="wikitable sortable">
<tbody><tr>
<th>Borough</th>
<th>Residents</th>
<th>Area</th>
<th>Number
</th></tr>
<tr>
<td><a href="/wiki/Alna" title="Alna">Alna</a></td>
<td>45 114</td>
<td>13,7 km<sup>2</sup></td>
<td align="right">12
</td></tr>
<tr>
<td><a href="/wiki/Bjerke" title="Bjerke">Bjerke</a></td>
<td>26 229</td>
<td>7,7 km<sup>2</sup></td>
<td align="right">9
</td></tr>
<tr>
<td><a href="/wiki/Frogner" title="Frogner">Frogner</a></td>
<td>47 618</td>
<td>8,3 km<sup>2</sup></td>
<td align="right">5
</td></tr>
<tr>
<td><a href="/wiki/Gamle_Oslo" title="Gamle Oslo">Gamle Oslo</a></td>
<td>39 500</td>
<td>7,5 km<sup>2</sup></td>
<td align="right">1
</td></tr>
<tr>
<td><a href="/wiki/Grorud" title="Grorud">Grorud</a></td>
<td>25 461</td>
<td>8,2 km<sup>2</sup></td>
<td align="right">10
</td></tr>
<tr>
<td><a href="/wiki/Gr%C3%BCnerl%C3%B8kka" title="Grünerløkka">Grünerløkka</a></td>
<td>42 129</td>
<td>4,8 km<sup>2</sup></td>
<td align="right">2
</td></tr>
<tr>
<td><a href

In [9]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Borough','Residents', 'Area', 'number']
df = pd.DataFrame(columns = column_names)

In [10]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==4:
        df.loc[len(df)] = row_data

In [11]:
df.head()

,Borough,Residents,Area,number
0,Alna,45 114,"13,7 km2",12
1,Bjerke,26 229,"7,7 km2",9
2,Frogner,47 618,"8,3 km2",5
3,Gamle Oslo,39 500,"7,5 km2",1
4,Grorud,25 461,"8,2 km2",10


From this table we create a list of the boroughs

In [12]:
list = df['Borough']
list

0                  Alna
1                Bjerke
2               Frogner
3            Gamle Oslo
4                Grorud
5           Grünerløkka
6           Nordre Aker
7            Nordstrand
8                Sagene
9        St. Hanshaugen
10              Stovner
11    Søndre Nordstrand
12               Ullern
13          Vestre Aker
14             Østensjø
Name: Borough, dtype: object

In [13]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
#!pip install folium
import folium
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


## Foursquare API. Searching businesess in the vicinity 

In [14]:
CLIENT_ID = '4UQCB3H1L3DLIZLOB3DFTKWR5TLCUYHVSJNN1VXOD5C50AM4' # your Foursquare ID
CLIENT_SECRET = 'CROXNP1UC1RN3RF5PT1YRH0YIKPWL03ENFUFNN2JGARHIYRL' # your Foursquare Secret
VERSION = '20200421'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4UQCB3H1L3DLIZLOB3DFTKWR5TLCUYHVSJNN1VXOD5C50AM4
CLIENT_SECRET:CROXNP1UC1RN3RF5PT1YRH0YIKPWL03ENFUFNN2JGARHIYRL


The purpose of the next code it is just to make sure that this code works properly.

In [15]:
address = 'Alna'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#latitudes.append(latitude, longitude)
print(latitude, longitude)

59.9324168 10.8352756


In [16]:
def get_location(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Oslo, Norway'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

From pgeocode, the database containing all the postal codes is downloaded.

In [17]:
import requests, zipfile, io

NO_url_zip='http://download.geonames.org/export/zip/NO.zip'
r = requests.get(NO_url_zip)
z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall()


We loop through all the neighbourhoods in order to obtain the exact location with Geolocator

In [18]:

latitudes=[]
longitudes=[]

for i in list:
    address = i

    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    latitudes.append(latitude)
    longitudes.append(longitude)

Now we have a list with the exact location of the neighbourhood. Look below.

In [19]:
df2 = pd.DataFrame()
df2['Neighborhood']=list
df2['Latitude']=latitudes
df2['Longitude']=longitudes
df2

,Neighborhood,Latitude,Longitude
0,Alna,59.932417,10.835276
1,Bjerke,59.941395,10.829208
2,Frogner,59.922224,10.706649
3,Gamle Oslo,59.899237,10.734767
4,Grorud,59.961424,10.880549
5,Grünerløkka,59.925471,10.777421
6,Nordre Aker,59.953638,10.756412
7,Nordstrand,54.487378,8.865286
8,Sagene,59.938273,10.765849
9,St. Hanshaugen,59.927950,10.738958


Now we can use Foursquare to get the business that we can find in every borough in Oslo. For this, we must create a function. 

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    radius=1000
    LIMIT=100
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=4UQCB3H1L3DLIZLOB3DFTKWR5TLCUYHVSJNN1VXOD5C50AM4&client_secret=CROXNP1UC1RN3RF5PT1YRH0YIKPWL03ENFUFNN2JGARHIYRL&v=20200214&ll=40.87655077879964,-73.91065965862981&radius=1000&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #results = requests.get(url, "none").json()["categories"]
                
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)

Now, we use the function defined above to obatin the data we are looking for. 

In [58]:
LIMIT = 500

Oslo_venues = getNearbyVenues(names=df2['Neighborhood'],
                              latitudes=df2['Latitude'],
                              longitudes=df2['Longitude']
                             )

Alna
Bjerke
Frogner
Gamle Oslo
Grorud
Grünerløkka
Nordre Aker
Nordstrand
Sagene
St. Hanshaugen
Stovner
Søndre Nordstrand
Ullern
Vestre Aker
Østensjø


We have obtained the business in the vinicity of every borough and the exact location of each one of them

In [59]:
print(Oslo_venues.shape)
Oslo_venues.head()

(1245, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alna,59.932417,10.835276,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Alna,59.932417,10.835276,Arturo's,40.874412,-73.910271,Pizza Place
2,Alna,59.932417,10.835276,Tibbett Diner,40.880404,-73.908937,Diner
3,Alna,59.932417,10.835276,Sam's Pizza,40.879435,-73.905859,Pizza Place
4,Alna,59.932417,10.835276,Starbucks,40.877531,-73.905582,Coffee Shop


In [60]:
Oslo_venues.groupby('Neighborhood').head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alna,59.932417,10.835276,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Alna,59.932417,10.835276,Arturo's,40.874412,-73.910271,Pizza Place
2,Alna,59.932417,10.835276,Tibbett Diner,40.880404,-73.908937,Diner
3,Alna,59.932417,10.835276,Sam's Pizza,40.879435,-73.905859,Pizza Place
4,Alna,59.932417,10.835276,Starbucks,40.877531,-73.905582,Coffee Shop
83,Bjerke,59.941395,10.829208,Bikram Yoga,40.876844,-73.906204,Yoga Studio
84,Bjerke,59.941395,10.829208,Arturo's,40.874412,-73.910271,Pizza Place
85,Bjerke,59.941395,10.829208,Tibbett Diner,40.880404,-73.908937,Diner
86,Bjerke,59.941395,10.829208,Sam's Pizza,40.879435,-73.905859,Pizza Place
87,Bjerke,59.941395,10.829208,Starbucks,40.877531,-73.905582,Coffee Shop


In [38]:
print('There are {} uniques categories.'.format(len(Oslo_venues['Venue Category'].unique())))

There are 52 uniques categories.


# Analyze each neighborhood

In [39]:
import pandas as pd

# one hot encoding
Oslo_onehot = pd.get_dummies(Oslo_venues[['Venue Category']], prefix="", prefix_sep="")
Oslo_onehot.head()

# add neighborhood column back to dataframe. Creating two dataframes and binding them together instead of using column list.

Oslo_onehot2 = pd.DataFrame(Oslo_venues['Neighborhood'])
Oslo_onehot2

df_c = pd.concat([Oslo_onehot2, Oslo_onehot], axis=1)

Oslo_onehot = df_c
Oslo_onehot.head()

#Oslo_onehot['Neighborhood'] = Oslo_venues['Neighborhood'] 

#cols = list(Oslo_onehot.columns)
#cols = [cols[-1]] + cols[:-1]
#df = Oslo_onehot[cols]


# move neighborhood column to the first column
#fixed_columns = [Oslo_onehot.columns[-1]] + list(Oslo_onehot.columns[:-1])
#Oslo_onehot = Oslo_onehot[fixed_columns]

#Oslo_onehot.head()


,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Bus Line,Bus Station,Café,Candy Store,...,Supermarket,Supplement Shop,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Video Game Store,Wine Shop,Yoga Studio
0,Alna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Alna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [40]:
Oslo_grouped = Oslo_onehot.groupby('Neighborhood').mean().reset_index()
Oslo_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Bus Line,Bus Station,Café,Candy Store,...,Supermarket,Supplement Shop,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Video Game Store,Wine Shop,Yoga Studio
0,Alna,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
1,Bjerke,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
2,Frogner,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
3,Gamle Oslo,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
4,Grorud,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
5,Grünerløkka,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
6,Nordre Aker,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
7,Nordstrand,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
8,Sagene,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048
9,St. Hanshaugen,0.024096,0.024096,0.024096,0.024096,0.012048,0.012048,0.012048,0.036145,0.012048,...,0.036145,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048,0.012048


**Let's confirm the new size**

In [41]:
Oslo_grouped.shape

(15, 53)

**Let's print each neighborhood along with the top 5 most common venues**

In [45]:
num_top_venues = 5

for hood in Oslo_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Oslo_grouped[Oslo_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alna----
                venue  freq
0                Park  0.07
1         Pizza Place  0.06
2      Sandwich Place  0.05
3                Café  0.04
4  Mexican Restaurant  0.04


----Bjerke----
                venue  freq
0                Park  0.07
1         Pizza Place  0.06
2      Sandwich Place  0.05
3                Café  0.04
4  Mexican Restaurant  0.04


----Frogner----
                venue  freq
0                Park  0.07
1         Pizza Place  0.06
2      Sandwich Place  0.05
3                Café  0.04
4  Mexican Restaurant  0.04


----Gamle Oslo----
                venue  freq
0                Park  0.07
1         Pizza Place  0.06
2      Sandwich Place  0.05
3                Café  0.04
4  Mexican Restaurant  0.04


----Grorud----
                venue  freq
0                Park  0.07
1         Pizza Place  0.06
2      Sandwich Place  0.05
3                Café  0.04
4  Mexican Restaurant  0.04


----Grünerløkka----
                venue  freq
0                Park  0

**Let's put that into a pandas dataframe**

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Oslo_grouped['Neighborhood']

for ind in np.arange(Oslo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Oslo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alna,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
1,Bjerke,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
2,Frogner,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
3,Gamle Oslo,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
4,Grorud,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
5,Grünerløkka,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
6,Nordre Aker,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
7,Nordstrand,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
8,Sagene,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout
9,St. Hanshaugen,Park,Pizza Place,Sandwich Place,Mexican Restaurant,Donut Shop,Café,Supermarket,Spanish Restaurant,American Restaurant,Scenic Lookout


# Cluster Neighborhoods

In [ ]:
Run k-means to cluster the neighborhood into 5 clusters.

In [51]:
# set number of clusters
kclusters = 5

Oslo_grouped_clustering = Oslo_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Oslo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

# Clusters visualization of Oslo neighborhoods

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Oslo_merged['Latitude'], Oslo_merged['Longitude'], Oslo_merged['Neighborhood'], Oslo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

### Cluster 2

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

### Cluster 3

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

### Cluster 4

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

### Cluster 5

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

# RESULTS

In this data it can be shown that the Frogner and Grunnerlorka are the areas in oslo which accumulates a higher number commercial businesess. By this reason it would be reccommended to invest there. 